# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = '../WeatherPy/output_data/clean_weather_df.csv'
cities_df = pd.read_csv(cities_csv)
cities_df = cities_df.drop('Unnamed: 0', 1)
cities_df = pd.DataFrame(cities_df)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Grand Island,40.9250,-98.3420,90.36,58,1,5.75,US,1627680884
1,Hong Kong,22.2855,114.1577,84.38,93,93,1.01,HK,1627680696
2,São Sebastião,-23.7600,-45.4097,54.28,73,87,3.56,BR,1627680680
3,Dongsheng,39.8161,109.9776,69.82,41,0,11.81,CN,1627680886
4,Vaini,-21.2000,-175.2000,73.56,94,90,8.05,TO,1627680886
...,...,...,...,...,...,...,...,...,...
583,Marapanim,-0.7175,-47.6997,76.42,89,18,4.18,BR,1627681238
584,Seredka,58.1584,28.1858,62.35,74,96,17.11,RU,1627681239
585,Paamiut,61.9940,-49.6678,46.44,83,100,7.27,GL,1627681048
586,Aleksandrov Gay,50.1470,48.5704,80.85,27,93,10.63,RU,1627681240


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities_df[['Lat','Lng']]
weight = cities_df['Humidity']

# Configuring gmaps
gmaps.configure(api_key=g_key)

# Adding layout parameters
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(
    layout=figure_layout,
    zoom_level=8,
    center=(34, -118),
    map_type='TERRAIN'
)

In [4]:
# Assign heat map to a variable
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=weight,
    max_intensity=100
)

# Add heat map layer to the map
fig.add_layer(heat_layer)

# Show map figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Maximum temperature of 80
ideal_cities_df = cities_df[cities_df['Max Temp'] <= 80]
# Humidity levels above 30
ideal_cities_df = ideal_cities_df[ideal_cities_df['Humidity']>= 30]
# Humidity levels below 50
ideal_cities_df = ideal_cities_df[ideal_cities_df['Humidity']<= 50]
# Maximum wind speeds of 20mph
ideal_cities_df = ideal_cities_df[ideal_cities_df['Wind Speed']<= 20]


# Drop null values
ideal_cities_df = ideal_cities_df.dropna()
ideal_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,Dongsheng,39.8161,109.9776,69.82,41,0,11.81,CN,1627680886
15,Thompson,55.7435,-97.8558,75.36,35,20,17.27,CA,1627680893
25,Mar del Plata,-38.0023,-57.5575,56.12,49,0,12.66,AR,1627680899
50,Khatanga,71.9667,102.5000,73.60,47,25,9.86,RU,1627680917
57,Yellowknife,62.4560,-114.3525,77.52,44,75,10.36,CA,1627680901


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into hotel_df
hotel_df = ideal_cities_df.loc[:, ["City", "Country", "Lat", "Lng"]]

# Add "Hotel Name" column to DataFrame
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
3,Dongsheng,CN,39.8161,109.9776,
15,Thompson,CA,55.7435,-97.8558,
25,Mar del Plata,AR,-38.0023,-57.5575,
50,Khatanga,RU,71.9667,102.5000,
57,Yellowknife,CA,62.4560,-114.3525,


In [7]:
# TESTING: what happens if no hotels are available near the city?
lat = '-7.2500'
lng = '35.0667'

params = {
    "location": f"{lat}, {lng}",
    "radius": 5000,
    "type": 'hotel',
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params).json()

pprint(response)

{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': -7.216666999999998,
                                        'lng': 35.05},
                           'viewport': {'northeast': {'lat': -7.205597266554719,
                                                      'lng': 35.06600742377141},
                                        'southwest': {'lat': -7.227736462638036,
                                                      'lng': 35.03399257622858}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'icon_background_color': '#7B9EB0',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_pinlet',
              'name': 'Kisanga',
              'place_id': 'ChIJgwPW-HcNUxgRp__tga55bkU',
              'reference': 'ChIJgwPW-HcNUxgRp__tga55bkU',
              'scope': 'GOOGLE',
              'types': ['locality', 'political'],
              'vicinity': 'Kisa

In [8]:
# TESTING: pulling name out of test json
response['results'][1]['name']

IndexError: list index out of range

In [9]:
# TESTING: script reads IndexError if no hotels are found near the city

In [10]:
# Assign Lat, Lng, and City into variables for each row of city
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city_name = row['City']
    
    # Assigning parameters
    params = {
        "location": f"{lat}, {lng}",
        "radius": 5000,
        "type": 'hotel',
        "key": g_key
    }
    
    # Requests.get
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params).json()
    try:
        hotel_name = response['results'][1]['name']
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = 'No hotel found'

In [13]:
hotel_df.head(5)

,City,Country,Lat,Lng,Hotel Name
3,Dongsheng,CN,39.8161,109.9776,Dongsheng Sports Center Experience Center
15,Thompson,CA,55.7435,-97.8558,Burntwood Hotel
25,Mar del Plata,AR,-38.0023,-57.5575,Gran Hotel Mar del Plata
50,Khatanga,RU,71.9667,102.5000,The prosecutor's office with. Hatanga
57,Yellowknife,CA,62.4560,-114.3525,Yellowknife Waterdrome


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer on top of heat map
marker_layer = gmaps.marker_layer(
    locations=locations,
    info_box_content=hotel_info
)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…